Developing an LSTM model to predict future stock prices involves several steps. 
These steps include data preprocessing, model building, hyperparameter tuning, and evaluation. 

Below, I'll outline the steps and provide code snippets for each part.

First, we need to collect and preprocess the historical stock price data. 

This typically involves normalizing the data and creating sequences for the LSTM model.